## Maximum Likelihood Estimation (1st & 2nd Moments) 

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.optimize import minimize
import yfinance as yfin

In [4]:
nvda_df = yfin.download('NVDA',start = '1990-01-01', end = '2024-07-01')['Adj Close'].pct_change().dropna()
nvda_df.head()

[*********************100%%**********************]  1 of 1 completed


Date
1999-01-25    0.104764
1999-01-26   -0.077594
1999-01-27   -0.003110
1999-01-28   -0.003144
1999-01-29   -0.047019
Name: Adj Close, dtype: float64

MLE: Finds the parameters of a function that are most likely to produce the observations seen, given an underlying probability distribution. It can be explained as fitting parameters to a probability distribution. By finding the combination of parameters that maximize the likelihood function, we can determine what the parameters should be given our observed data. This involves finding the critical point or when gradient = 0. 

PDF of Normal Distribution (Stock Returns) 
$$
f(x \mid \mu, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left( -\frac{(x - \mu)^2}{2\sigma^2} \right)
$$

Log Likelihood Function: (Summation represent the joint probability distribution) 

$$
\ell(\mu, \sigma \mid x) = \sum_{i=1}^{n} \log \left( \frac{1}{\sqrt{2\pi\sigma^2}} \right) + \sum_{i=1}^{n} \log \left( \exp\left( -\frac{(x_i - \mu)^2}{2\sigma^2} \right) \right)
$$

Simplifying: 
$$
-\ell(\mu, \sigma \mid x) = \frac{n}{2} \log (2\pi\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^{n} (x_i - \mu)^2
$$

Then find the critical points, derivate with respect to mu and set it equal to 0.

In [62]:
def log_likelihood(parameters, dataset):
    mu, sigma = parameters 
    n = len(dataset)
    term1 = -n/2 * np.log(2*np.pi*sigma**2)
    term2 = np.sum((dataset - mu)**2)/(2*sigma**2) 
    likelihood = term1 - term2 
    return -likelihood

init_guess = [.001,.001]
result = minimize(log_likelihood, init_guess, args=(nvda_df.values), method='L-BFGS-B', bounds=[(None, None), (1e-6, None)])
display(result)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: -11868.288787324154
        x: [ 1.977e-03  3.787e-02]
      nit: 8
      jac: [-3.871e+00 -3.378e-01]
     nfev: 87
     njev: 29
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [66]:
MLE_moment1 = result.x[0]
MLE_moment2 = result.x[1]

print(f'MLE Sample Mean: {MLE_moment1}')
print(f'MLE Sigma: {MLE_moment2}')

MLE Sample Mean: 0.001976748909181311
MLE Sigma: 0.037867972165235436


In [65]:
print(f'Actual Sample Mean {nvda_df.mean()}')
print(f'Actual Sample Sigma {nvda_df.std()}')

Actual Sample Mean 0.0019776213749344573
Actual Sample Sigma 0.03787097423598338
